In [ ]:
%load_ext autoreload
%autoreload 2

from csv import DictWriter

import torch.cuda
from ultralytics import YOLO
from omegaconf import OmegaConf

from digital_wave import (
    PROJ_ROOT, DATA_DIR, CONFIG_DIR, MODELS_DIR,
)

In [ ]:
config_fname = "train.yaml"

config_path = CONFIG_DIR / config_fname
cfg = OmegaConf.load(config_path)

In [ ]:
config_data_fpath = DATA_DIR / cfg.dataset_name / "data.yaml"
save_exp_dir = PROJ_ROOT / cfg.save_exp_dir

In [ ]:
model = YOLO(cfg.model_name)

### Train

In [ ]:
train_params = {
    "data": config_data_fpath,
    "project": save_exp_dir,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    **cfg.model,
}

model.train(**train_params)

### Test

In [ ]:
best_model_path = MODELS_DIR / "yolov11n_best.pt"
best_model = YOLO(best_model_path)

In [ ]:
test_params = {
    "project": save_exp_dir,
    "split": "test",
}

torch.cuda.empty_cache()
results = best_model.val(**test_params)

In [ ]:
metrics_csv_path = results.save_dir / "results.csv"

with open(metrics_csv_path, "w") as csvfile: 
    writer = DictWriter(csvfile, fieldnames=results.results_dict) 
    writer.writeheader() 
    writer.writerows([results.results_dict]) 